# My XGBoost implementation

This time, plan is a little diffrent. I will exceptionally not implement the code from scratch, because I would probably need C++ efficiency.
Main goal of these mini-projects was to have handy and not complicated "notebook" written in Python to refresh my knowledge later in my journey. XGBoost is more computationally difficult, so it would disrupt the initial assumption for this project.

Of course, I could implement it on smaller dataset, but does it make sense when XGBoost was made for really large datasets?

So, this time I will do only two steps.

    1. Notes for XGBoost
    2. Implementation from libary

In [22]:
print("Let's get to work!")

Let's get to work!


## 1. Notes
    A. Main idea
    B. How it works
    C. Math behind the algorithm
    D. When to use it?

### A. Main idea

**XGBoost** is an algorithm that builds many decision trees, where <u> each new tree corrects the errors of the previous ones.</u> <br>
It is based on Gradient Boosting, but introduces optimizations that make it faster and more effective on large and complex datasets. <br>

Key improvments are:

    * Regularization to reduce overfitting -> adds λ penalties to prevent overfitting and improve generalization.
  
    * Efficient handling of missing values -> automatically learns the best path for missing data, no need for preprocessing.
  
    * Parallelization for speed  -> computes tree splits in parallel, making training much faster on large datasets.
  
    * Sparsity awareness and memory optimization -> optimized for sparse data and memory usage with efficient data structures.


### B. How it works

1) Start with an initial prediction (e.g., 0.5 for log-loss classification, or the mean of y for regression).
2) Calculating <u>Residuals</u> for every example (Residual = prediction - real_data).
3) Building first leaf that contains all training samples.
4) For the first leaf, we calculate <u>Similarity Score</u>
   
   ![](Sim_score.png)

   in easier language...


   ![](sim_score_2.png) 

5) After that, we split the residuals into two groups in many possible ways. Suppose we have m residuals: in the first scenario, one residual goes to the left leaf and the remaining m-1 to the right; in the next scenario, two residuals go left and m-2 go right; and so on.
6) For each scenario we calculate <u>Gain</u>
   
   ![](gain.png) 

   ![](gain2.png)

   We chceck gain in every scenario, and pick the biggest one. The winner becomes a new node.

7) This pattern continues, making new nodes until it reaches the max depth.

8) However, we pick γ - parameter for prunning tree. Now we check if gain - γ is negative or not. If it is, we prune a node, and check if prevous node is also negative.

9) Calculating output values
    
    ![](output_val.png)


10) Update predictions: new prediction = old prediction + 𝜂⋅ (leaf weight), where 𝜂 is the learning rate.

    ![](n_pred.png)

quick summary at 23:57 minute -> https://www.youtube.com/watch?v=OtD8wVaFm6E

### C. Algorithm in Math

### D. When to use it?

* ✅ When you have large and complex datasets

* ✅ When data contains missing values

* ✅ When you need high accuracy 

* ✅ Works with diffrent types of data

## 2. Implementation from library

In [23]:
# Importing libraries

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import xgboost as xgb
import numpy as np



In [24]:
# Importing dataset 

df = pd.read_csv(r"C:\Users\wikto\OneDrive\Dokumenty\all-datasets\Lung Cancer.csv")

# Prepering Data

df = df.drop('id', axis=1)

# 1. Getting days passed since diagnosis
df["diagnosis_date"] = pd.to_datetime(df["diagnosis_date"])
df["end_treatment_date"] = pd.to_datetime(df["end_treatment_date"])

treatment_time = (df["diagnosis_date"] - df["end_treatment_date"]).abs()

df["treatment_time"] = treatment_time
df["treatment_time"] = df["treatment_time"].dt.days.astype('int')

df = df.drop(["diagnosis_date", "end_treatment_date"], axis=1)


# 2. Fixing category type of data

df["gender"] = df["gender"].astype('category')
df["country"] = df["country"].astype('category')
df["cancer_stage"] = df["cancer_stage"].astype('category')
df["smoking_status"] = df["smoking_status"].astype('category')
df["treatment_type"] = df["treatment_type"].astype('category')
df["family_history"] = df["family_history"].astype('category')


# 3. Data Normalization

# Normalizacja nie miała dla XGBoosta żadnego znazenia 
scal = MinMaxScaler()
df[["age"]] = scal.fit_transform(df[["age"]])
df[["bmi"]] = scal.fit_transform(df[["bmi"]])
df[["cholesterol_level"]] = scal.fit_transform(df[["cholesterol_level"]])
df[["treatment_time"]] = scal.fit_transform(df[["treatment_time"]])

# Splitting to X's and y
y = df["survived"]
X = df.drop("survived", axis=1)

#X = pd.get_dummies(X, drop_first=False)

# Splitting to train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

df.head()


,age,gender,country,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,survived,treatment_time
0,0.60,Male,Sweden,Stage I,Yes,Passive Smoker,0.462069,0.326667,0,0,1,0,Chemotherapy,0,0.621572
1,0.46,Female,Netherlands,Stage III,Yes,Passive Smoker,0.868966,0.866667,1,1,0,0,Surgery,1,0.440585
2,0.61,Female,Hungary,Stage III,Yes,Former Smoker,0.965517,0.786667,1,1,0,0,Combined,0,0.341865
3,0.47,Female,Belgium,Stage I,No,Passive Smoker,0.931034,0.606667,1,1,0,0,Chemotherapy,0,0.475320
4,0.33,Male,Luxembourg,Stage I,No,Passive Smoker,0.127586,0.186667,0,0,0,0,Combined,0,0.407678


In [25]:
# Converting dataset into DMatrix structure
xgb_train = xgb.DMatrix(X_train, y_train, enable_categorical=True)
xgb_test = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [ ]:
# Creating XGBoost Model

# 1. Picking parameters

params = {
    'objective': 'binary:logistic', # type of learning task
    'max_depth': 4,  # maximum depth level of a tree
    'learning_rate': 0.01,
    'tree_method': 'hist', # How trees are built - 'hist' uses histogram-based algorithm (faster on large datasets)
    'scale_pos_weight':3.56, # it improved recall, precision and F1-score <- scales according to how much more often 0's occur, usefull when data is inbalanced
    'min_child_weight': 8, # minimum number of weights that can lay in single leaf -  prevents overfitting by requiring more samples per leaf
    # 'early_stopping_rounds': 50 # after 50 round without mutch change in loss function, trainning stops
}
n = 1000 # number of maximum iterations
model = xgb.train(params=params, dtrain=xgb_train, num_boost_round=n)


c:\Users\wikto\OneDrive\Dokumenty\Implementations of ML\XGBoost\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:28:23] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "early_stopping_rounds" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


### XGBoost – wpływ wielkości parametrów i zalecane zakresy

| Parametr              | Mała wartość                                                                 | Duża wartość                                                                 | Zalecany zakres startowy |
|-----------------------|-------------------------------------------------------------------------------|-------------------------------------------------------------------------------|---------------------------|
| **objective**         | Nie dotyczy skali (binary:logistic = klasyfikacja binarna, output 0-1).      | Nie dotyczy.                                                                 | 'binary:logistic'         |
| **max_depth**         | Płytsze drzewa, prostsze reguły, mniejsze ryzyko przeuczenia, ale możliwe niedouczenie. | Głębsze drzewa, większa zdolność dopasowania, większe ryzyko przeuczenia, wolniejsze trenowanie. | 3 – 8 (często 4–6)        |
| **learning_rate**     | Wolniejsze uczenie, stabilniejsze, potrzeba więcej drzew, zwykle lepsza generalizacja. | Szybsze uczenie, mniej drzew, większe ryzyko przeuczenia i niestabilności.   | 0.01 – 0.1                |
| **tree_method**       | Algorytm budowy drzew (`hist` = szybki, oszczędny w pamięci). Skala nie dotyczy. | Skala nie dotyczy.                                                           | 'hist' lub 'approx'       |
| **scale_pos_weight**  | Klasy traktowane jako w przybliżeniu zbalansowane.                           | Większe znaczenie klasy rzadkiej, rośnie recall, ale może spaść precision.   | ≈ (liczba_neg / liczba_pos) |
| **min_child_weight**  | Liście mogą powstawać z małą liczbą próbek – model elastyczny, większe ryzyko przeuczenia. | Wymaga więcej próbek w liściu – silniejsza regularyzacja, mniej podziałów, mniejsze ryzyko przeuczenia. | 1 – 10                    |
| **early_stopping_rounds** | Szybsze zatrzymanie, ryzyko zbyt wczesnego stopu przy szumiących danych.   | Dłuższe trenowanie, mniejsze ryzyko przedwczesnego zatrzymania.              | 20 – 100                  |
| **num_boost_round**   | Mało drzew – model może być niedouczony (szczególnie przy niskim learning_rate). | Dużo drzew – większa pojemność modelu, ryzyko przeuczenia (chyba że używasz early stopping). | 500 – 2000 (z early stopping) |

---

### Interakcje
- Niższy `learning_rate` + wyższy `num_boost_round` = częsta bezpieczna kombinacja.  
- Wyższy `max_depth` zwykle wymaga wyższego `min_child_weight` lub dodatkowej regularyzacji.  
- `scale_pos_weight` dobieraj wg proporcji klas, potem koryguj pod precision/recall.  


In [27]:
# Checking model precision, recall, f1-score
y_pred = model.predict(xgb_test)
y_pred = np.round(y_pred)

class_report = classification_report(y_test, y_pred)

print(f'{class_report}')

              precision    recall  f1-score   support

           0       0.78      0.49      0.60    138962
           1       0.22      0.51      0.30     39038

    accuracy                           0.49    178000
   macro avg       0.50      0.50      0.45    178000
weighted avg       0.66      0.49      0.54    178000



As you can see, the model does not perform well on this type of data. Problem probably lies in the very small number of samples containing survived patients. Also, due to the quite big amount of categorical data, CatBoost would likely perform better than XGBoost.